In [31]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [32]:
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

In [33]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"
)

In [34]:
sample_documents = [
    {
        "title":"Kota Bandung: Paris van Java yang Selalu Menawan",
        "content":"""Bandung adalah ibu kota Provinsi Jawa Barat yang terkenal dengan julukan “Paris van Java” karena keindahan alam dan gaya hidup masyarakatnya yang modern. Terletak di dataran tinggi, Bandung memiliki udara yang sejuk dan pemandangan pegunungan yang indah, menjadikannya salah satu destinasi wisata favorit di Indonesia.
Selain keindahan alamnya, Bandung juga dikenal sebagai kota kreatif. Banyak anak muda yang berkreasi di bidang fashion, kuliner, dan teknologi. Kawasan seperti Dago, Braga, dan Cihampelas menjadi pusat kegiatan ekonomi kreatif, sementara kampus ternama seperti Institut Teknologi Bandung (ITB) menjadikan kota ini sebagai tempat lahirnya banyak inovator.
Bandung juga memiliki banyak tempat wisata menarik, seperti Kawah Putih, Gunung Tangkuban Parahu, Lembang, dan Farmhouse. Tak hanya itu, wisata kuliner Bandung pun sangat terkenal. Makanan seperti batagor, surabi, seblak, dan peuyeum menjadi ikon khas yang wajib dicoba oleh wisatawan.
Dengan keindahan alam, budaya kreatif, dan kuliner yang menggugah selera, Bandung tetap menjadi kota yang selalu dirindukan. Baik untuk berlibur, belajar, maupun berkarya, Bandung selalu punya pesona yang sulit dilupakan."""
    },
    {
        "title":"Pulau Bali: Surga Wisata di Indonesia",
        "content":"""
        Bali adalah salah satu pulau terkenal di Indonesia yang sering disebut sebagai Pulau Dewata. Pulau ini dikenal di seluruh dunia karena keindahan alamnya, budaya yang unik, serta keramahan penduduknya. Setiap tahun, jutaan wisatawan datang ke Bali untuk menikmati pantainya, pegunungannya, dan keindahan tradisi masyarakatnya.
Salah satu daya tarik utama Bali adalah pantainya yang indah, seperti Kuta, Sanur, Nusa Dua, dan Pandawa. Selain itu, kawasan Ubud menjadi pusat seni dan budaya Bali, dengan pemandangan sawah hijau, galeri seni, dan pertunjukan tari tradisional.
Budaya Bali juga sangat kaya dan terjaga hingga kini. Upacara keagamaan, tarian, serta arsitektur pura menjadi bagian penting dalam kehidupan masyarakatnya. Salah satu acara terkenal adalah Hari Nyepi, hari raya umat Hindu di Bali yang dirayakan dengan penuh kedamaian dan keheningan.
Selain wisata alam dan budaya, Bali juga berkembang sebagai destinasi wisata modern. Banyak hotel, restoran, dan kafe bergaya internasional berdiri, namun tetap harmonis dengan budaya lokal.
Dengan keindahan alam, budaya yang kuat, dan keramahan masyarakatnya, Bali layak disebut sebagai permata pariwisata Indonesia yang memikat hati siapa pun yang datang."""
    }
]

In [35]:
# potong-potong / chunking
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

In [37]:
from langchain_core.documents import Document

In [38]:
document = []

for doc in sample_documents:
    doc = Document(page_content=doc["content"], metadata={"title": doc["title"]})
    document.append(doc)

In [39]:
document

[Document(metadata={'title': 'Kota Bandung: Paris van Java yang Selalu Menawan'}, page_content='Bandung adalah ibu kota Provinsi Jawa Barat yang terkenal dengan julukan “Paris van Java” karena keindahan alam dan gaya hidup masyarakatnya yang modern. Terletak di dataran tinggi, Bandung memiliki udara yang sejuk dan pemandangan pegunungan yang indah, menjadikannya salah satu destinasi wisata favorit di Indonesia.\nSelain keindahan alamnya, Bandung juga dikenal sebagai kota kreatif. Banyak anak muda yang berkreasi di bidang fashion, kuliner, dan teknologi. Kawasan seperti Dago, Braga, dan Cihampelas menjadi pusat kegiatan ekonomi kreatif, sementara kampus ternama seperti Institut Teknologi Bandung (ITB) menjadikan kota ini sebagai tempat lahirnya banyak inovator.\nBandung juga memiliki banyak tempat wisata menarik, seperti Kawah Putih, Gunung Tangkuban Parahu, Lembang, dan Farmhouse. Tak hanya itu, wisata kuliner Bandung pun sangat terkenal. Makanan seperti batagor, surabi, seblak, dan pe

In [40]:
chunks = text_splitter.split_documents(document)

In [41]:
chunks

[Document(metadata={'title': 'Kota Bandung: Paris van Java yang Selalu Menawan'}, page_content='Bandung adalah ibu kota Provinsi Jawa Barat yang terkenal dengan julukan “Paris van Java” karena keindahan alam dan gaya hidup masyarakatnya yang modern. Terletak di dataran tinggi, Bandung memiliki udara yang sejuk dan pemandangan pegunungan yang indah, menjadikannya salah satu destinasi wisata favorit di Indonesia.'),
 Document(metadata={'title': 'Kota Bandung: Paris van Java yang Selalu Menawan'}, page_content='Selain keindahan alamnya, Bandung juga dikenal sebagai kota kreatif. Banyak anak muda yang berkreasi di bidang fashion, kuliner, dan teknologi. Kawasan seperti Dago, Braga, dan Cihampelas menjadi pusat kegiatan ekonomi kreatif, sementara kampus ternama seperti Institut Teknologi Bandung (ITB) menjadikan kota ini sebagai tempat lahirnya banyak inovator.'),
 Document(metadata={'title': 'Kota Bandung: Paris van Java yang Selalu Menawan'}, page_content='Bandung juga memiliki banyak tem

In [42]:
# membangun vector store dengan FAISS

from langchain_community.vectorstores import FAISS

In [43]:
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

In [44]:
# membuat retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [45]:
user_query = "apa wisata paling indah di bandung?"

relevant_docs = retriever.invoke(user_query)

In [46]:
relevant_docs[0].page_content

'Dengan keindahan alam, budaya kreatif, dan kuliner yang menggugah selera, Bandung tetap menjadi kota yang selalu dirindukan. Baik untuk berlibur, belajar, maupun berkarya, Bandung selalu punya pesona yang sulit dilupakan.'

In [47]:
context = "\n\n".join([doc.page_content for doc in relevant_docs])

In [48]:
context

'Dengan keindahan alam, budaya kreatif, dan kuliner yang menggugah selera, Bandung tetap menjadi kota yang selalu dirindukan. Baik untuk berlibur, belajar, maupun berkarya, Bandung selalu punya pesona yang sulit dilupakan.\n\nBandung juga memiliki banyak tempat wisata menarik, seperti Kawah Putih, Gunung Tangkuban Parahu, Lembang, dan Farmhouse. Tak hanya itu, wisata kuliner Bandung pun sangat terkenal. Makanan seperti batagor, surabi, seblak, dan peuyeum menjadi ikon khas yang wajib dicoba oleh wisatawan.\n\nBandung adalah ibu kota Provinsi Jawa Barat yang terkenal dengan julukan “Paris van Java” karena keindahan alam dan gaya hidup masyarakatnya yang modern. Terletak di dataran tinggi, Bandung memiliki udara yang sejuk dan pemandangan pegunungan yang indah, menjadikannya salah satu destinasi wisata favorit di Indonesia.'

In [49]:
from langchain_core.prompts import ChatPromptTemplate

In [50]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant that provides accurate information based on the context provided
    
    context: {context}
    """),
    ("human", "{user_query}")
])

In [51]:
prompt = prompt_template.format(
    context=context,
    user_query=user_query
)

In [52]:
prompt

'System: \n    You are a helpful assistant that provides accurate information based on the context provided\n\n    context: Dengan keindahan alam, budaya kreatif, dan kuliner yang menggugah selera, Bandung tetap menjadi kota yang selalu dirindukan. Baik untuk berlibur, belajar, maupun berkarya, Bandung selalu punya pesona yang sulit dilupakan.\n\nBandung juga memiliki banyak tempat wisata menarik, seperti Kawah Putih, Gunung Tangkuban Parahu, Lembang, dan Farmhouse. Tak hanya itu, wisata kuliner Bandung pun sangat terkenal. Makanan seperti batagor, surabi, seblak, dan peuyeum menjadi ikon khas yang wajib dicoba oleh wisatawan.\n\nBandung adalah ibu kota Provinsi Jawa Barat yang terkenal dengan julukan “Paris van Java” karena keindahan alam dan gaya hidup masyarakatnya yang modern. Terletak di dataran tinggi, Bandung memiliki udara yang sejuk dan pemandangan pegunungan yang indah, menjadikannya salah satu destinasi wisata favorit di Indonesia.\n    \nHuman: apa wisata paling indah di ba

In [53]:
response = llm.invoke(prompt)
response.content

'Berdasarkan informasi yang diberikan, beberapa tempat wisata indah di Bandung antara lain:\n\n*   Kawah Putih\n*   Gunung Tangkuban Parahu\n*   Lembang\n*   Farmhouse'